In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


Identyfikator sesji w GA4 nie jest unikalny . Jest to sygnatura czasowa rozpoczęcia sesji, więc wielu użytkowników może (i często ma) mieć ten sam identyfikator sesji przypisany do jednej z ich sesji.

Aby faktycznie stworzyć unikalny identyfikator sesji, możesz użyć CONCAT, aby połączyć user_pseudo_id i ga_session_id . Ponieważ każdy użytkownik może rozpocząć tylko jedną sesję o określonej sygnaturze czasowej, ta kombinacja zawsze da unikalny identyfikator.

Sesja jest uważana za zaangażowaną, jeśli jakiekolwiek działanie w tej sesji ma parametr session_engaged ustawiony na wartość 1.

In [ ]:
%%bigquery --project 

select
    extract(year from timestamp_micros(user_first_touch_timestamp)) as year_first_touch,
    count(distinct user_pseudo_id) as users
from
    `ga.events_*`

group by 
    1
having 
    year_first_touch is not null
order by
    year_first_touch desc

,year_first_touch,users
0,2021,43133740
1,2020,11236805


Count Regular Sessions Using ga_session_id

In [ ]:
%%bigquery --project 

select
    count(distinct (select value.int_value from unnest(event_params) where key = 'ga_session_id')) as sessions_only_session_id,
    count(distinct concat(user_pseudo_id,(select value.int_value from unnest(event_params) where key = 'ga_session_id'))) as sessions_session_and_user_id
from 
    `ga.events_20210601`

,sessions_only_session_id,sessions_session_and_user_id
0,6733,7102


In [ ]:
%%bigquery --project 

select
    concat(user_pseudo_id,'.',(select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'ga_session_id')) as true_session_id,
from
    `ga.events_*`
where
    _table_suffix between '20210805' and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by 
    1
order by
    1 desc

,true_session_id
0,zyWcn8TRPjidm7Fo+2dJO//FQ8UL5Io9566CqZk7VBE=.1...
1,zyPBg3tWYfjUGXLM3SvZGHLN6uyOoSqQ88EdxjOZnXQ=.1...
2,zyPBg3tWYfjUGXLM3SvZGHLN6uyOoSqQ88EdxjOZnXQ=.1...
3,zyPBg3tWYfjUGXLM3SvZGHLN6uyOoSqQ88EdxjOZnXQ=.1...
4,zyPBg3tWYfjUGXLM3SvZGHLN6uyOoSqQ88EdxjOZnXQ=.1...
...,...
102242,+2fLSZpdNz9op+rM0rAxdP6yiGi5M6vcsOZ5uwFAK2c=.1...
102243,+1OyDZ8FfKesmc63OLA1qr8XBn+hfAE93NSMPxVq480=.1...
102244,+0hQgt0DVhMmxT4ba6MesedCYNbMY4QEpqvcndlQKkA=.1...
102245,+0hQgt0DVhMmxT4ba6MesedCYNbMY4QEpqvcndlQKkA=.1...


Explore Engagement Using session_engaged

Sesje z zaangażowaniem: liczba sesji, które trwały co najmniej 10 sekund albo w których zarejestrowano co najmniej 1 zdarzenie konwersji lub co najmniej 2 wyświetlenia strony.

https://support.google.com/analytics/answer/9191807?hl=pl


Funkcja MAX służy do odszukania najwyższej niepustej wartości, jaką w danej sesji miała sesja dla session_engaged

In [ ]:
%%bigquery --project 

select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    max((select value.string_value from unnest(event_params) where key = 'session_engaged')) as session_engaged
from
    `ga.events_20210601`
group by 
    user_pseudo_id,
    session_id

,user_pseudo_id,session_id,session_engaged
0,1836464821.1622549536,1622549535,1
1,1519925207.1622560592,1622560591,0
2,636502471.1622551047,1622551046,1
3,1119237899.1616481074,1622536644,0
4,1119237899.1616481074,1622560951,0
...,...,...,...
7097,1762092380.1622507151,1622507150,0
7098,1762092380.1622507151,1622509623,0
7099,2027077558.1622521264,1622521263,1
7100,693452447.1622524545,1622524544,1


Count Engaged Sessions Using session_engaged

In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    max((select value.string_value from unnest(event_params) where key = 'session_engaged')) as session_engaged
from
    `ga.events_20210601`
group by 
    user_pseudo_id,
    session_id)

select 
    count(distinct concat(user_pseudo_id,session_id)) as sessions,
    count(distinct case when session_engaged = '1' then concat(user_pseudo_id,session_id) else null end) as engaged_sessions
from 
    prep

,sessions,engaged_sessions
0,7102,4416


Count Engaged Users Using engagement_time_msec

ms - milisekundy

In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    max((select value.string_value from unnest(event_params) where key = 'session_engaged')) as session_engaged,
    max((select value.int_value from unnest(event_params) where key = 'engagement_time_msec')) as engagement_time_msec
from
    `ga.events_20210601`
group by 
    user_pseudo_id,
    session_id)

select 
    count(distinct concat(user_pseudo_id,session_id)) as sessions,
    count(distinct case when session_engaged = '1' then concat(user_pseudo_id,session_id) else null end) as engaged_sessions,
    count(distinct user_pseudo_id) as users,
    count(distinct case when engagement_time_msec > 0 then user_pseudo_id else null end) as active_users
from 
    prep

,sessions,engaged_sessions,users,active_users
0,7102,4416,5013,4418


Explore Session Traffic Sources

In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    max((select value.int_value from unnest(event_params) where key = 'engagement_time_msec')) as engagement_time_msec,
    max((select value.string_value from unnest(event_params) where key = 'session_engaged')) as session_engaged,
    max((select value.string_value from unnest(event_params) where key = 'medium')) as medium,
    max((select value.string_value from unnest(event_params) where key = 'source')) as source,
from
    `ga.events_20210601`
group by
    user_pseudo_id,
    session_id)

select
    concat(ifnull(source,'(direct)'),' / ',ifnull(medium,'(none)')) as session_source_medium,
    count(distinct concat(user_pseudo_id,session_id)) as sessions,
    count(distinct case when session_engaged = '1' then concat(user_pseudo_id,session_id) end) as engaged_sessions,
    count(distinct user_pseudo_id) as users,
    count(distinct case when engagement_time_msec > 0 then user_pseudo_id end) as active_users
from
    prep
group by 
    session_source_medium
order by 
    sessions desc
LIMIT 3

,session_source_medium,sessions,engaged_sessions,users,active_users
0,google / organic,4236,2836,3158,2885
1,(direct) / (none),1785,924,1307,1009
2,convertkit / email,446,277,337,296


Rekord jest prostym typem pola zagnieżdżonego . Rekord jest reprezentowany przez właściwości i wartości zagnieżdżone w nazwie pola wyższego poziomu.

In [ ]:
%%bigquery --project 

select
    concat(traffic_source.source,' / ',traffic_source.medium) as source_medium,
    traffic_source.source,
    traffic_source.medium,
    traffic_source.name as campaign,
    count(distinct user_pseudo_id) as users
from
    `ga.events_20210601`
group by 
    source_medium,
    source,
    medium,
    campaign
order by
    users desc
LIMIT 2

,source_medium,source,medium,campaign,users
0,google / organic,google,organic,(organic),3324
1,(direct) / (none),(direct),(none),(direct),1013


In [ ]:
%%bigquery --project 

# Tam gdzie jest iPhone daj null
select
    user_pseudo_id,
    nullif(device.mobile_model_name, 'iPhone') device
from
    `ga.events_20210601`
where 
    device.category = 'mobile'
group by 
    1,2
LIMIT 5

,user_pseudo_id,device
0,1836464821.1622549536,Redmi 5
1,1519925207.1622560592,Mi Note 10 Lite
2,636502471.1622551047,SM-G981U1
3,1897084720.1622569459,None
4,1042623904.1581967789,None


In [ ]:
%%bigquery --project 

# Tam gdzie jest pusty znak daj NULL

select
    geo.continent,
    geo.country,
    nullif(geo.city,'') as city,
    count(distinct user_pseudo_id) as users
from
    `ga.events_20210607`
group by 
    continent,
    country,
    city
order by 
    users desc
LIMIT 8

,continent,country,city,users
0,Europe,United Kingdom,London,114
1,Europe,Germany,Berlin,62
2,Europe,Sweden,Stockholm,58
3,Europe,France,Paris,56
4,Oceania,Australia,Sydney,53
5,Europe,United Kingdom,None,48
6,Europe,Spain,Madrid,47
7,Asia,South Korea,Seoul,44
